In [72]:
import requests
from xml.dom.minidom import parseString

url = "https://int44.zakupki.gov.ru/eis-integration/services/getDocsIP" #correct для ФЛ

token = "3735f424-f2ac-4e0b-9c26-7f5b69e5c04a" #correct
idx = "83a5e824-4d1d-41b0-aba3-82417a4baa43" #correct
time = "2024-09-02T10:00:00" #correct?
mode = "PROD" 
sbsType = "PRIZ" #correct
reestr_number = "0338200005124000169" # correct - artem
#resstr_number = "0338300047925000002"

soapEnvelope = f"""
<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:ws="http://zakupki.gov.ru/fz44/get-docs-ip/ws">
    <soapenv:Header>
        <individualPerson_token>{token}</individualPerson_token>
    </soapenv:Header>
    <soapenv:Body>
        <ws:getDocsByReestrNumberRequest>
            <index>
                <id>{idx}</id>
                <createDateTime>{time}</createDateTime>
                <mode>{mode}</mode>
            </index>
            <selectionParams>
                <subsystemType>{sbsType}</subsystemType>
                <reestrNumber>{reestr_number}</reestrNumber>
            </selectionParams>
        </ws:getDocsByReestrNumberRequest>
    </soapenv:Body>
</soapenv:Envelope>

"""
options = {
    "Content-Type": "text/xml; charset=utf-8"
}
response = requests.post(url, data = soapEnvelope, headers = options)
response.text

'<?xml version="1.0" encoding="UTF-8"?><soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">   <soap:Header/>   <soap:Body>      <ns2:getDocsByReestrNumberResponse xmlns:ns2="http://zakupki.gov.ru/fz44/get-docs-ip/ws">         <index>            <id>5bea0ced-904b-462c-9e42-be93b91bd18f</id>            <refId>83a5e824-4d1d-41b0-aba3-82417a4baa43</refId>            <createDateTime>2025-01-27T15:38:43.451</createDateTime>            <mode>PROD</mode>         </index>         <dataInfo>            <archiveUrl>https://int44.zakupki.gov.ru/dstore/ip/download/PRIZ/file.zip?ticket=L%2BIHZLPT%2B8E4CWy3Z8nj9Zvf5SVbMVAYbuVQE4Taag8GEGlI%2FnZKqLeB5OAmNS%2BhlOvLBg%2FlpsTds5kROcHbNLHyzXjhXYKOoQDo4vv8W16cfV8H%2FOpwJIXhag%2FqfOVg3t1hItIKGgqvXLZiwTufJvu2jLqQgnBgFbwAd4CZpTZDRgV7fM4U%2Fb1lU0VpmTZjotrQxGU8jP1TVqPE8Qb0M2PSNK%2FJ47qwzJ0VMBIziDZxil7H7IcTAnVDrnwHV8PiPw7vWXLqNWGyIsLXIAWpr4DL7pNWdpAl%2Bhx8RFJ8sXpYR1dXIQa3cr5nXxK8fRsyHEkepJosD6982JF0vRVnVg%3D%3D</archiveUrl>         </dataInfo>  

In [59]:
# with open("response.xml", "w", encoding="utf-8") as file:
#     file.write(response.text)

In [60]:
import requests

url = "https://int44.zakupki.gov.ru/dstore/ip/download/PRIZ/file.zip?ticket=A6r7KIVhshemWkbs92E83oaKRM0QMwY4dwR2vANrW0p5fN%2Bt%2FFbFW4SZor%2FS5p2UQlTYydxJvBNGWmXR8avhadFrc8V4zbdyxLfh5RyUb%2FmvO79BdenvGiMpWslqT28bCz%2BEEVFt8OZpAYWY0fKsnDrhyEUp8rA2CGQugsD6mljvyXhqkTD6NWfaL42i7A43THXQW5Ayb2Dtst9OEQ8ZXVGViqY452vA6we9K2sJteW2lnhFTb%2B9BfWtpsMso9wk0nXI3vrg%2BULhl9hBfwboVZardZK5kNrLkZkOGdLxK2U2vwgPBGveR3jyFfb2XnBDzkBbY4sEEhFeUzxcA606ag%3D%3D"
#url = "https://int44.zakupki.gov.ru/dstore/ip/download/PRIZ/file.zip"
headers = {
    "individualPerson_token": "3735f424-f2ac-4e0b-9c26-7f5b69e5c04a"
}
response = requests.get(url, headers=headers)

In [69]:
with open("file.zip", "wb") as f:
    f.write(response.content)

import zipfile
with zipfile.ZipFile("file.zip", "r") as zip_ref:
    zip_ref.extractall("output_directory")

import os
os.remove('file.zip')

In [70]:
import xml.etree.ElementTree as ET
filename = "./output_directory/epNotificationEZK2020_0338200005124000169_1.xml"
tree = ET.parse(filename)
root = tree.getroot()

import shutil
shutil.rmtree('output_directory')

namespaces = {
    "ns3": "http://zakupki.gov.ru/oos/export/1",
    "ns5": "http://zakupki.gov.ru/oos/EPtypes/1"
}
purchase_number = root.find(".//ns5:purchaseNumber", namespaces).text
purchase_object_info = root.find(".//ns5:purchaseObjectInfo", namespaces).text
responsible_org_info_fullname = root.find(".//ns5:responsibleOrgInfo/ns5:fullName", namespaces).text

res = {
    'externalId':purchase_number,
    'query':purchase_object_info
}
res

{'externalId': '0338200005124000169',
 'query': 'Оказание услуг по техническому обслуживанию систем (автоматическая пожарной сигнализации (АПС) и система оповещения и управления эвакуацией (СОУЭ) людей при пожаре'}